# Converting Timestamps to Minutes Per Hour

The following attempts to take non-uniform timestamps and do the following:
* resample those timestamps into minutes
* forward-fill the states
* resample the minutes into hours, summing the minutes

The function `.resample()` changed in pandas 0.18.1, so this was a learning process.

In [1]:
import numpy as np
import pandas as pd
print "This should be '0.20.1':"
print "pandas:         " + str(pd.__version__)
print "This should be '1.12.1':"
print "numpy:          " + str(np.__version__)

This should be '0.20.1':
pandas:         0.20.3
This should be '1.12.1':
numpy:          1.13.1


importing the data from the Library data sample.

In [2]:
useData = pd.read_csv(r'../data/170830_StateData.csv')

In [3]:
useData.head()

,computerName,dateStamp,state
0,DMC0021,2017-08-31 00:00:46.393,offline
1,DMC0014,2017-08-31 00:00:46.450,offline
2,DMC0009,2017-08-31 00:00:46.480,offline
3,CRR028,2017-08-31 00:00:46.507,offline
4,CRR017,2017-08-31 00:00:46.510,offline


This is an interesting exercise demonstrating that there are occurrences that evaluated to having occurred at the exact same millisecond.

In [4]:
useData[pd.to_datetime(useData.dateStamp).duplicated(keep=False)]

,computerName,dateStamp,state
181,DMC0016,2017-08-31 06:30:28.460,restarted
182,CRR017,2017-08-31 06:30:28.460,restarted
836,DDL0001,2017-08-31 10:57:09.637,offline
837,TL7001,2017-08-31 10:57:09.637,available


Verifying the status of the columns. 'dataStamp' should be a datetime64 field.

In [5]:
useData.dateStamp = useData.dateStamp.apply(pd.to_datetime)

In [6]:
useData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 3 columns):
computerName    1049 non-null object
dateStamp       1049 non-null datetime64[ns]
state           1049 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 24.7+ KB


Testing functionality with a single computer. Working with iterating across the DataFrame later.

In [7]:
computerDataName = 'CITI002'

In [8]:
computerTimeArray = useData[useData.computerName == computerDataName]

In [9]:
computerTimeArray

,computerName,dateStamp,state
40,CITI002,2017-08-31 00:05:23.990,offline
88,CITI002,2017-08-31 00:06:00.460,restarted
499,CITI002,2017-08-31 09:30:04.523,in-use
791,CITI002,2017-08-31 10:52:06.430,available
792,CITI002,2017-08-31 10:52:07.003,offline
799,CITI002,2017-08-31 10:52:53.107,restarted


This turns the 'in-use' value into true, and all of the others into false. Since this analysis is based upon when a machine is not being used (as opposed to when it is offline/available/restarted) all other states are irrelevant.

Will need to investigate copy/view on this error. May need to do this as a dataframe with `.concat()`.

In [10]:
computerTimeArray.loc[:,'state'] = pd.Series(computerTimeArray.state == 'in-use')

/Users/brown/Envs/CSC693/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Location data is irrelevant for a machine at this point. Also, location can be derived from machine name, machine location is (at this point) not that precise.

The duplicates with regard to the indexing is no longer an issue. Since the dateStamp field before only had 'minute' precision, this has been fixed by importing data with 'millisecond' precision.

In [11]:
computerTimeArray

,computerName,dateStamp,state
40,CITI002,2017-08-31 00:05:23.990,False
88,CITI002,2017-08-31 00:06:00.460,False
499,CITI002,2017-08-31 09:30:04.523,True
791,CITI002,2017-08-31 10:52:06.430,False
792,CITI002,2017-08-31 10:52:07.003,False
799,CITI002,2017-08-31 10:52:53.107,False


In [12]:
computerTimeArray = computerTimeArray.set_index('dateStamp').sort_index()

In [13]:
computerTimeArray

,computerName,state
dateStamp,,
2017-08-31 00:05:23.990,CITI002,False
2017-08-31 00:06:00.460,CITI002,False
2017-08-31 09:30:04.523,CITI002,True
2017-08-31 10:52:06.430,CITI002,False
2017-08-31 10:52:07.003,CITI002,False
2017-08-31 10:52:53.107,CITI002,False


This takes the above data and resamples it into minute increments. Value for the specific minute is put into place, while 'NaN' values will take on the previous non-NaN data.

In [14]:
computerTimeArrayMin = computerTimeArray.resample('T').ffill()

In [15]:
computerTimeArrayPerHour = computerTimeArrayMin.resample('H').sum()

In [16]:
computerTimeArrayPerHour

,state
dateStamp,
2017-08-31 00:00:00,0
2017-08-31 01:00:00,0
2017-08-31 02:00:00,0
2017-08-31 03:00:00,0
2017-08-31 04:00:00,0
2017-08-31 05:00:00,0
2017-08-31 06:00:00,0
2017-08-31 07:00:00,0
2017-08-31 08:00:00,0


This is a matrix of the datestamp times as the index, computers as the columns, and the state change at the intersection.

In [17]:
fullMatrix = useData.pivot(index='dateStamp',columns='computerName',values='state').sort_index()

In [18]:
fullMatrix

computerName,BL001,BL002,CITI001,CITI002,CITI003,CITI004,CITI005,CITI006,CITI007,CITI008,...,TL4004,TL5001,TL5002,TL6002,TL7001,TL702,TL801,TL802,TL901,TL902
dateStamp,,,,,,,,,,,,,,,,,,,,,
2017-08-31 00:00:46.393,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.450,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.480,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.507,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.510,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.540,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.550,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.557,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-08-31 00:00:46.560,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


While this gets the dataframe into the preferred format, it causes problems with resampling later. It appears that the resampling method looks at the value at the first datestamp that matches a particular minute. If there are multiple entries at that minute, it takes the value of the first one, and applies it for the whole minute.

In [19]:
dateTestRange = pd.date_range(start='01-01-2017',end='01-02-2017',freq='s')

In [20]:
dateTestSeries = pd.Series(dateTestRange)

In [21]:
dateTestDataFrame = pd.concat([dateTestSeries,pd.Series()],axis=1)

In [22]:
dateTestDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86401 entries, 0 to 86400
Data columns (total 2 columns):
0    86401 non-null datetime64[ns]
1    0 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.3 MB


In [23]:
dateTestDataFrame.columns = ['DateTime','StateAtTime']

In [24]:
dateTestDataFrame.iloc[14,1] = True

In [25]:
dateTestDataFrame.set_index('DateTime',inplace=True)

In [26]:
dateTestDataFrame.head(15)

,StateAtTime
DateTime,
2017-01-01 00:00:00,NaN
2017-01-01 00:00:01,NaN
2017-01-01 00:00:02,NaN
2017-01-01 00:00:03,NaN
2017-01-01 00:00:04,NaN
2017-01-01 00:00:05,NaN
2017-01-01 00:00:06,NaN
2017-01-01 00:00:07,NaN
2017-01-01 00:00:08,NaN


In [27]:
dateTestDataFrame.resample('H',how=np.any)

/Users/brown/Envs/CSC693/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """Entry point for launching an IPython kernel.


,StateAtTime
DateTime,
2017-01-01 00:00:00,NaN
2017-01-01 01:00:00,NaN
2017-01-01 02:00:00,NaN
2017-01-01 03:00:00,NaN
2017-01-01 04:00:00,NaN
2017-01-01 05:00:00,NaN
2017-01-01 06:00:00,NaN
2017-01-01 07:00:00,NaN
2017-01-01 08:00:00,NaN


In [28]:
def inUseConvert(state):
    if state == 'in-use':
        return 1
    else:
        return 0